In [1]:
from othello_rules import *
from othello_net import *
from tensorflow.python.framework import ops
from datetime import datetime
from example_states import *
from feature_extractor import *
from training_utils import *
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
def value_error(data_path, sess, diagnostics=False):
    errors = []
    validation_matches = get_all_matches(data_path)
    #XXX: Delete this line when testing is faster
    validation_matches = validation_matches[0:10]
    for i in range(len(validation_matches)):
        if diagnostics:
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>new game")
        match = validation_matches[i]
        raw_match_movelist = match[8:]
        unpacked_movelist = unpack('b'*60, raw_match_movelist)
        black_score = match[6]
        unpacked_black_score = unpack('b', black_score)
        winner = -1 if unpacked_black_score[0] > 32 else 1
        board = initialize_game()
        player = -1
        
        if diagnostics:
            print(unpacked_black_score)
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>winner is " + str(winner))
        movenr = 0
        for move in unpacked_movelist:
            if move == 0:
                break
            feature_path = 'cache/validation/features/features_' + str(i) + "_" + str(move) + ".npy"
            label_path = 'cache/validation/labels/labels_' + str(i) + "_" + str(move) + ".npy"
            if os.path.isfile(feature_path) and os.path.isfile(label_path):
                features = np.load(feature_path)
            else:
                features = board_to_input(board, player)
                
            input_batch = [features]
            #s = 1 if player * (-1) == winner else 0
            if winner == -1:
                r = ([0, 1])
                #r = [[0]]
            else:
                r = ([1, 0])
                #r = [[1]]
            label_batch = [r]
            

            if diagnostics and movenr > 55:
                estimate = sess.run(score_out, feed_dict={img_data:input_batch, keep_prob:1.0})
                predictions = sess.run(pred_up, feed_dict={img_data:input_batch, keep_prob:1.0})
                predictions = np.array(predictions[0])
                predictions = predictions / np.sum(predictions)
                print("-new state-")
                print("actual value: " + str(r))
                #print("score_out: " + str(estimate))
                print("predictions: " + str(predictions))
            movenr += 1
            if np.count_nonzero(board) == 64:
                print("ok got some")
                error = sess.run(loss, feed_dict={img_data:input_batch, ground_truths: label_batch, keep_prob:1.0})
                errors.append(error)
            board = make_move(board, move, player)
            if player is 1:
                player = -1
            else:
                player = 1
            legal_moves = find_legal_moves(board, player)
            if len(legal_moves) == 0:
                if player is 1:
                    player = -1
                else:
                    player = 1
                    
        if np.count_nonzero(board) == 64:
            error = sess.run(loss, feed_dict={img_data:input_batch, ground_truths: label_batch, keep_prob:1.0})
            errors.append(error)
            print("target is ")
            print(r)
            predic = sess.run(score_out, feed_dict={img_data:input_batch, ground_truths: label_batch, keep_prob:1.0})
            predic = predic[0][0][0]
            prediction = list_softmax(predic)
            print("prediction is:")
            print(prediction)
    return np.sum(errors) / len(errors)

def list_softmax(x):
    """Compute softmax values for each sets of scores in x."""
    sf = np.exp(x)
    sf = sf/np.sum(sf, axis=0)
    return sf

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def lrelu(x, leak=0.05, name="lrelu"):
    return tf.maximum(x, leak*x)

def conv_nonparams2(tensor_input, conv_weights, conv_biases, keep_prob):
    conv_in = conv(tensor_input, conv_weights)
    conv_relu = lrelu(conv_in + conv_biases)
    conv_drop = tf.nn.dropout(conv_relu, keep_prob)
    return conv_drop

def conv_weights2(i, o):
    k = 2
    shape = [k, k, i, o]
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def conv_nonparams3(tensor_input, conv_weights, conv_biases, keep_prob):
    conv_in = conv(tensor_input, conv_weights, "VALID")
    conv_relu = lrelu(conv_in + conv_biases)
    conv_drop = tf.nn.dropout(conv_relu, keep_prob)
    return conv_drop

def create_value_net():
    with tf.name_scope("value_network"):
        img_data = tf.placeholder(tf.float32, shape=[None, 8, 8, 28], name="img_data")
        keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        #convolutional layers 
        i = 28; o = 32;
        conv1_weights = conv_weights(i, o)
        conv1_biases = conv_biases(o)
        conv1_out = conv_nonparams2(img_data, conv1_weights, conv1_biases, keep_prob)
        #pool1 = max_pool_2x2(conv1_out)
        i = o; o = 32;
        conv2_weights = conv_weights(i, o)
        conv2_biases = conv_biases(o)
        conv2_out = conv_nonparams2(conv1_out, conv2_weights, conv2_biases, keep_prob)
        #pool2 = max_pool_2x2(conv2_out)
        i = o; o = 32;
        conv3_weights = conv_weights2(i, o)
        conv3_biases = conv_biases(o)
        conv3_out = conv_nonparams2(conv2_out, conv3_weights, conv3_biases, keep_prob)
        
        k = 8; i = o; o = 2;
        score_weights = weight_variable([k, k, i, o])
        score_biases = bias_variable([o])
        score_out = conv_nonparams3(conv3_out, score_weights, score_biases, keep_prob)
        score_out_flat = tf.reshape(score_out, [-1, o])
        print(score_out_flat.get_shape())
        print(score_out.get_shape())
        #final layer
        #Don't actually use this layer here
        predictions = softmax(score_out, o)

        #training block:
        label = tf.placeholder(tf.float32, shape=[None, 2], name="ground_truths")
        print(label.get_shape())
        learn_rate = tf.placeholder(tf.float32, name="eta")
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(score_out_flat, label))
        policy_network_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        reg_loss = tf.reduce_sum([tf.reduce_sum(tf.square(x)) for x in policy_network_variables])
        optimizer = tf.train.AdamOptimizer(learn_rate)
        train_step = optimizer.minimize(loss)
        sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
        return sess.graph, img_data, train_step, optimizer, label, loss, predictions, keep_prob, learn_rate, score_out

    # MAKE SURE that the topology of this network is the same as the policy network!

In [3]:
ops.reset_default_graph()
graph, img_data, train_step, optimizer, ground_truths, loss, pred_up, keep_prob, learn_rate, score_out = create_value_net()
saver = tf.train.Saver()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
init_op = tf.initialize_all_variables()
sess.run(init_op)
#nopool7 gat fundið hver sigrar leiki, 0.2~ loss
current_model = "models/sl-v/nopool12.ckpt"
if os.path.isfile(current_model):
    saver.restore(sess, current_model)
validation_path = "validation/"
matches = get_all_matches('training/')
lenmatches = len(matches)
print(lenmatches)

# Byrjum þjálfunina
print("start training")
#print("starting error:" + str(avg_error(validation_path, sess)))
#print('%s: Step %d: Prediction accuracy = %.2f' % (datetime.now(), 0,
#                                                      prediction_accuracy()/float(60)))
diagn = False
if diagn:
    iterations = 1
else:
    iterations = 1
prev_stop = 0
probs = 1
start = 1e-9
for i in range(prev_stop, prev_stop+iterations):
    #TODO: Skrifa þetta fall
    #input_batch, label_batch = prepare_train_batch(train_ids, batch_size, do_flips, do_rots, data_path)
    current_match = matches[i]
    raw_match_movelist = current_match[8:]
    unpacked_movelist = unpack('b'*60, raw_match_movelist)
    black_score = current_match[6]
    unpacked_black_score = unpack('b', black_score)
    
    #XXX: Temporary check for the value network, do this properly later
    if unpacked_black_score[0] > 32:
        winner = -1
    else:
        winner = 1
    
    board = initialize_game()
    training_stability = np.zeros((8,8))
    player = -1
    
    # One training batch is all the data from one match
    input_batch = []
    label_batch = []
    
    for move in unpacked_movelist:
        if move == 0:
            break
        feature_path = 'cache/training/features/features_' + str(i) + "_" + str(move) + ".npy"
        label_path = 'cache/training/labels/labels_' + str(i) + "_" + str(move) + ".npy"
        if os.path.isfile(feature_path) and os.path.isfile(label_path):
            try:
                features = np.load(feature_path)
                label = np.load(label_path)
            except:
                print("data corruption in match " + str(i))
                features = board_to_input(board, player)
                label = prepare_data(move_to_label(move))
        else:
            features = board_to_input(board, player)
            label = prepare_data(move_to_label(move))
        #s = 1 if player * (-1) == winner else 0
        if winner == -1:
            #r = [[0]]
            r = ([0, 1])
        else:
            #r = [[1]]
            r = ([1, 0])
        #print(r)
        #if np.random.rand()>0.0:
        if False:
            print(features)
            input_batch.append(features)
            label_batch.append(r)
            label_batch.append(r)
            label_batch.append(r)
            label_batch.append(r)
            # Now we add 3 reflections of the game state
            # which is done by flipping the board over one diagonal        
            move_upright = flip_move_upright(move)
            features_upright = flip_features(features, 'upright')
            input_batch.append(features_upright)
            # Then the other diagonal
            move_upleft = flip_move_upleft(move)
            features_upleft = flip_features(features, 'upleft')
            input_batch.append(features_upleft)
            # Then both diagonals
            move_both = flip_move_upright(flip_move_upleft(move))
            features_both = flip_features(features, 'both')
            input_batch.append(features_both)
        board = make_move(board, move, player)
        if player is 1:
            player = -1
        else:
            player = 1
        legal_moves = find_legal_moves(board, player)
        if len(legal_moves) == 0:
            if player is 1:
                player = -1
            else:
                player = 1
    #print(features)
    input_batch.append(features)
    label_batch.append(r)
    eta = start
    #label_batch = [label_batch]
    #label_batch = np.array(label_batch)
    #print(label_batch.shape)
    #print(np.count_nonzero(board) == 64)
    if np.count_nonzero(board) == 64 and len(input_batch) > 0:
        #print("training!")
        #print(label_batch)
        _, loss_ = sess.run([train_step, loss],
                               feed_dict={img_data:input_batch,
                                                ground_truths: label_batch,
                                                keep_prob:probs,
                                                learn_rate:eta})
    #print("score out is")
    #print(loss_)

    if (i % 100 is 0) and (i > 0) or (i+1) == (iterations+prev_stop) or (i % 1000 is 0):  
        print('%s, Step %d, Loss = %.6f' % (datetime.now().strftime("%d. %b %H:%M:%S"), i,
                                                                          value_error(validation_path, sess, diagn)))
        save_path = saver.save(sess, current_model)

print("done")

(?, 2)
(?, 1, 1, 2)
(?, 2)
117298
start training
target is 
[1, 0]
prediction is:
[ 0.67418522  0.32581478]
target is 
[1, 0]
prediction is:
[ 0.80082536  0.19917466]
target is 
[1, 0]
prediction is:
[ 0.95703542  0.04296459]
target is 
[1, 0]
prediction is:
[ 0.83524984  0.16475013]
target is 
[0, 1]
prediction is:
[ 0.03645425  0.96354574]
target is 
[0, 1]
prediction is:
[ 0.32411849  0.67588151]
target is 
[1, 0]
prediction is:
[ 1.  0.]
target is 
[1, 0]
prediction is:
[ 0.73553765  0.26446232]
target is 
[1, 0]
prediction is:
[ 0.99975663  0.00024339]
target is 
[1, 0]
prediction is:
[ 0.70647943  0.29352057]
14. Oct 16:57:29, Step 0, Loss = 0.192403
done


In [4]:
# always hanging around 0.693147
# "models/sl-v/tolf.ckpt" náði 0.55975 í fyrstu þjálfun!
# start =6e-5 virðist láta tolf-derivation lækka um 0.05 per 50 steps...
# Þegar ég hækka learning rate þá virðist allt ýtast niðður í 0.5f

In [5]:
print(8*8*236)

15104


In [6]:
board1 = initialize_game()
zeros = np.count_nonzero(board1)
print(zeros)

4


(?, 2)
(?, 1, 1, 2)
(?, 2)
117298
start training

First 1e-3 down to 0.4, then:

start = 1e-4

start training
14. Oct 16:55:09, Step 0, Loss = 0.349850
14. Oct 16:55:15, Step 100, Loss = 0.338470
14. Oct 16:55:20, Step 200, Loss = 0.296196
14. Oct 16:55:25, Step 300, Loss = 0.283280
14. Oct 16:55:31, Step 400, Loss = 0.310574
14. Oct 16:55:36, Step 500, Loss = 0.308730
14. Oct 16:55:41, Step 600, Loss = 0.269805
14. Oct 16:55:47, Step 700, Loss = 0.255041
14. Oct 16:55:52, Step 800, Loss = 0.255202
14. Oct 16:55:57, Step 900, Loss = 0.233139
14. Oct 16:56:03, Step 1000, Loss = 0.230880
14. Oct 16:56:08, Step 1100, Loss = 0.192403